In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import json
import csv
import pandas as pd
import urllib.request as urlReq
import wget


In [2]:
def searchSome(query,number=100):
    imagesPerPage = 100
    jsonFiles = []
    firstQuery = min(number,imagesPerPage)
    url = urlReq.urlopen("https://www.rijksmuseum.nl/api/en/collection/?ps="+str(firstQuery)+"&q=Q&key=t2KR0CWi&format=json"
                         .replace("Q", query))
    jsonfile = json.loads(url.read().decode())
    myCsv = jsonCSV(jsonfile)
    if(number>jsonfile['count']):
        raise ValueError("Illegal Argument: maximum value for param number is " + str(jsonfile['count']))
    if(number>imagesPerPage): 
        number = number - imagesPerPage
        index = 2
        while number > 0 :
            
            url = urlReq.urlopen("https://www.rijksmuseum.nl/api/en/collection/?p="+str(index)+"&ps=100&q=Q&key=t2KR0CWi&format=json"
                                 .replace("Q", query))
            if(min(number,imagesPerPage)!=imagesPerPage):
                jsonFiles.append(jsonCSV(json.loads(url.read().decode()))[0:number])
            else :
                jsonFiles.append(jsonCSV(json.loads(url.read().decode())))                

            number = number - imagesPerPage
            index +=1
        for csv in jsonFiles:
            myCsv = pd.concat((myCsv,csv),ignore_index = True)
            
    return myCsv


#Can take a long time for big ste of images
def searchAll(query):
    url = urlReq.urlopen("https://www.rijksmuseum.nl/api/en/collection/?q=Q&key=t2KR0CWi&format=json"
                         .replace("Q", query))
    jsonfile = json.loads(url.read().decode())
    return searchSome(query,jsonfile['count'])
    
def jsonCSV(file):
    df = pd.DataFrame.from_dict(file['artObjects'])
    return df
def downloadImages(csv, directory):
    invalid = 0
    for i in range(csv.shape[0]):
        if(csv['webImage'][i] is not None):
            wget.download(csv['webImage'][i]['url'],out=directory+"/image"+str(i)+".jpg")
        else :
            invalid += 1
    return invalid
            
    

In [68]:
def getDatesFromCSV(csv):
    idDate = {}
    ids = csv["id"]
    for elem in ids:
        elem = elem.upper()[3:]
        url = urlReq.urlopen("https://www.rijksmuseum.nl/api/nl/collection/"+elem+"?key=t2KR0CWi&format=json")
        jsonfile = json.loads(url.read().decode())
        idDate[elem] = jsonfile["artObject"]["dating"]['sortingDate']
    return idDate;
    

        

In [69]:
mCsv= searchSome('rembrandt',20)


In [71]:
dates = getDatesFromCSV(mCsv)